In [1]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
# os.environ['HADOOP_USER_NAME']='student02'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as F
from pyspark import StorageLevel

In [2]:
conf = SparkConf().setAppName('Persist_Checkpoint_20240527')\
    .setMaster('yarn') \
    .set('spark.sql.shuffle.partitions', '10')\
    .set("spark.eventLog.logBlockUpdates.enabled", 'true')

spark = SparkSession.builder.config(conf=conf) \
    .getOrCreate()

sc = spark.sparkContext
print("app_id".ljust(50), sc.applicationId)
# print("spark.sql.warehouse.dir".ljust(50), spark.conf.get("spark.sql.warehouse.dir"))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/27 19:50:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/27 19:50:39 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
24/05/27 19:50:39 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


app_id                                             application_1711527309433_0362


In [3]:
import random
from datetime import date, timedelta

end = date.today()
start = end + timedelta(days=-300)

names = ['Bob', 'James', 'Marek', 'Johannes', 'Иванов', 'Смирнов', 'Кузнецов', 'Попов', 'Васильев', 'Петров', 'Соколов', 'Михайлов', 'Новиков', 'Федоров', 'Морозов', 'Волков', 'Алексеев', 'Лебедев', 'Семенов', 'Егоров', 'Павлов', 'Козлов', 'Степанов', 'Николаев']
countries = ['rus:eu', 'bel:eu', 'usa:am', 'fra:eu', 'aus:as', 'ger:eu', 'swe:eu', 'fin:eu', 'chi:as']
customers = list()

sc.setJobDescription('MY DESCRIPTION JOB 1')

In [34]:
length_customers = 100000
customers = list()
for i in range(length_customers):
    random_date = start + (end - start) * random.random()

    if i % 3 == 0:
        country = countries[-1] # chi - 'China'
    else:
        country = countries[random.randint(0, len(countries)-2)] # without China
    
    customer = (names[random.randint(0, len(names)-1)], country, random.randint(-1000000, 1000000), random_date)
    customers.append(customer)

df_customers = spark.sparkContext.parallelize(customers, 5)\
    .toDF(schema=["name","country","balance","last_order_date"])

In [44]:
print(df_customers.rdd.getNumPartitions())
df_customers_cached = df_customers.persist(StorageLevel.DISK_ONLY)
# df_customers_cached.rdd.setName("DF Customers Cached")
df_customers_cached.count()

5


100000

In [ ]:
sc.setCheckpointDir("/user/ssenigov/")
df_customers_checkpointed = df_customers.checkpoint()

In [43]:
# df_customers.write.saveAsTable("tblCustomers1", mode='overwrite')
df_customers_cached.unpersist()

DataFrame[name: string, country: string, balance: bigint, last_order_date: date]

In [ ]:
 --
spark.stop()